In [ ]:
#install tensorflow
!pip install tensorflow

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models
#from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#import the dataset
data = pd.read_csv("UNSW_NB15_testing-set.csv")
data
data.shape

(175341, 45)

In [ ]:
#find missing values in data
data.isnull().sum()

,0
id,0
dur,0
proto,0
service,0
state,0
spkts,0
dpkts,0
sbytes,0
dbytes,0
rate,0


In [ ]:
#check datatype of each column
data.dtypes

,0
id,int64
dur,float64
proto,object
service,object
state,object
spkts,int64
dpkts,int64
sbytes,int64
dbytes,int64
rate,float64


In [ ]:
#get unique values in protocol column
data["proto"].unique()

array(['tcp', 'udp', 'arp', 'ospf', 'icmp', 'igmp', 'rtp', 'ddp',
       'ipv6-frag', 'cftp', 'wsn', 'pvp', 'wb-expak', 'mtp', 'pri-enc',
       'sat-mon', 'cphb', 'sun-nd', 'iso-ip', 'xtp', 'il', 'unas',
       'mfe-nsp', '3pc', 'ipv6-route', 'idrp', 'bna', 'swipe',
       'kryptolan', 'cpnx', 'rsvp', 'wb-mon', 'vmtp', 'ib', 'dgp',
       'eigrp', 'ax.25', 'gmtp', 'pnni', 'sep', 'pgm', 'idpr-cmtp',
       'zero', 'rvd', 'mobile', 'narp', 'fc', 'pipe', 'ipcomp', 'ipv6-no',
       'sat-expak', 'ipv6-opts', 'snp', 'ipcv', 'br-sat-mon', 'ttp',
       'tcf', 'nsfnet-igp', 'sprite-rpc', 'aes-sp3-d', 'sccopmce', 'sctp',
       'qnx', 'scps', 'etherip', 'aris', 'pim', 'compaq-peer', 'vrrp',
       'iatp', 'stp', 'l2tp', 'srp', 'sm', 'isis', 'smp', 'fire', 'ptp',
       'crtp', 'sps', 'merit-inp', 'idpr', 'skip', 'any', 'larp', 'ipip',
       'micp', 'encap', 'ifmp', 'tp++', 'a/n', 'ipv6', 'i-nlsp',
       'ipx-n-ip', 'sdrp', 'tlsp', 'gre', 'mhrp', 'ddx', 'ippc', 'visa',
       'secure-vmtp', 

In [ ]:
# #label encode the categorical columns objects
# label_encoder = LabelEncoder()
# for column in data.select_dtypes(include=["object"]).columns:
#     data[column] = label_encoder.fit_transform(data[column])
#     print(column)

# data['proto'].unique()
# data.shape

In [ ]:
# Data preprocessing function
def preprocess_data(data):
    data = data.drop(["id", "attack_cat"], axis=1, errors="ignore")  # Drop irrelevant columns
    label_encoder = LabelEncoder()
    for column in data.select_dtypes(include=["object"]).columns:
        data[column] = label_encoder.fit_transform(data[column])
    scaler = MinMaxScaler()
    data[data.columns] = scaler.fit_transform(data)
    return data

train_data = data
# test_data = pd.read_csv("UNSW_NB15_training-set.csv")

# Preprocess training data
X_train = preprocess_data(train_data.drop("label", axis=1))
y_train = train_data["label"]

# X_test = preprocess_data(test_data.drop("label", axis=1))
# y_test = test_data["label"]

In [ ]:
# test_data = pd.read_csv("UNSW_NB15_training-set.csv")
# X_test = preprocess_data(test_data.drop("label", axis=1))
# y_test = test_data["label"]

In [ ]:
X_train.shape

(175341, 42)

In [ ]:
#build autoencoder and extract features
# Define input layer with 45 features
input_dim = 42
latent_dim = 20

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(64, activation='relu')(input_layer)  # You can adjust the hidden layer size
encoded = layers.Dense(latent_dim, activation='relu')(encoded)

# Decoder
decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

# Autoencoder model
autoencoder = models.Model(input_layer, decoded)

# Encoder model (to get the 20-dimensional representation)
encoder = models.Model(input_layer, encoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse')  # Mean Squared Error loss for reconstruction


In [ ]:
X_train.shape

(175341, 42)

In [ ]:
# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.0605 - val_loss: 0.0014
Epoch 2/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0019 - val_loss: 4.1429e-04
Epoch 3/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 7.3548e-04 - val_loss: 3.0465e-04
Epoch 4/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 5.6526e-04 - val_loss: 2.6493e-04
Epoch 5/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4.9700e-04 - val_loss: 2.1236e-04
Epoch 6/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.3410e-04 - val_loss: 1.8862e-04
Epoch 7/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.8834e-04 - val_loss: 1.5639e-04
Epoch 8/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.4242e-04 - val_loss: 1.4830e-04
Epoch 9/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.3156e-04 - val_loss: 1.3352e-04
Epoch 10/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.8375e-04 - val_loss: 1.5303e-04
Epoch 11/50
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.7369e-04

In [ ]:
# Get the 20 features for the training data
X_encoded = encoder.predict(X_train)


5480/5480 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step


In [ ]:
type(X_encoded)

numpy.ndarray

In [ ]:
#add encoded features to x_train forn prediction
X_train = pd.concat([X_train, pd.DataFrame(X_encoded)], axis=1)
X_train.shape

(175341, 62)

In [ ]:
#get datatype of all columns
X_train.dtypes.unique()

array([dtype('float64'), dtype('float32')], dtype=object)

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
#import random forest classifier and train on it
X_train.columns = X_train.columns.astype(str)
rf_classifier = RandomForestClassifier(n_estimators=256, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=256, random_state=42)

In [ ]:
test_data = pd.read_csv("UNSW_NB15_training-set.csv")
X_test = preprocess_data(test_data.drop("label", axis=1))
y_test = test_data["label"]

In [ ]:
#test on test data and give classification report
X_test_encoded = encoder.predict(X_test)
X_test = pd.concat([X_test, pd.DataFrame(X_test_encoded)], axis=1)
X_test.columns = X_test.columns.astype(str)
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
              precision    recall  f1-score   support

           0       0.99      0.62      0.76     37000
           1       0.76      0.99      0.86     45332

    accuracy                           0.83     82332
   macro avg       0.88      0.81      0.81     82332
weighted avg       0.86      0.83      0.82     82332



In [ ]:
#other classification algorithms like logisitic regression and svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#fit to svm with rbf kernel
svm_classifier = SVC(kernel='rbf', probability=True)
X_train.columns = X_train.columns.astype(str)
svm_classifier.fit(X_train, y_train)
#test on test data and give classification report
X_test_encoded = encoder.predict(X_test)
X_test = pd.concat([X_test, pd.DataFrame(X_test_encoded)], axis=1)
X_test.columns = X_test.columns.astype(str)
y_pred_svm = svm_classifier.predict(X_test)
print(classification_report(y_test, y_pred_svm))

In [ ]:
#create stacking layers with base as random forest and svm
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True))
]

stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression()
)

stacking_model.fit(X_train, y_train)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('svm', SVC(probability=True))],
                   final_estimator=LogisticRegression())

In [ ]:
#predict on stacking model and get classification report
y_pred_stacking = stacking_model.predict(X_test_encoded)
print(classification_report(y_test, y_pred_stacking))


              precision    recall  f1-score   support

           0       0.74      0.93      0.82     56000
           1       0.96      0.84      0.90    119341

    accuracy                           0.87    175341
   macro avg       0.85      0.89      0.86    175341
weighted avg       0.89      0.87      0.87    175341

